# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 13 2023 12:07PM,259179,19,9259589,"Snap Medical Industries, LLC",Released
1,Mar 13 2023 12:07PM,259179,19,9260649,"Snap Medical Industries, LLC",Released
2,Mar 13 2023 12:07PM,259178,18,EMERSA-9260650,"Emersa Waterbox, LLC",Released
3,Mar 13 2023 11:40AM,259175,15,9259593,"Brookfield Pharmaceuticals, LLC",Released
4,Mar 13 2023 11:40AM,259175,15,9259596,"Brookfield Pharmaceuticals, LLC",Released
5,Mar 13 2023 11:40AM,259175,15,9259597,"Brookfield Pharmaceuticals, LLC",Released
6,Mar 13 2023 11:40AM,259175,15,9259598,"Brookfield Pharmaceuticals, LLC",Released
7,Mar 13 2023 11:40AM,259175,15,9259599,"Brookfield Pharmaceuticals, LLC",Released
8,Mar 13 2023 11:40AM,259175,15,9259600,"Brookfield Pharmaceuticals, LLC",Released
9,Mar 13 2023 11:40AM,259175,15,9260128,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,259173,Released,1
27,259174,Released,9
28,259175,Released,24
29,259178,Released,1
30,259179,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259173,NaN,NaN,1.0
259174,NaN,NaN,9.0
259175,NaN,NaN,24.0
259178,NaN,NaN,1.0
259179,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259125,0.0,0.0,10.0
259138,1.0,12.0,2.0
259141,0.0,1.0,0.0
259145,0.0,0.0,23.0
259148,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259125,0,0,10
259138,1,12,2
259141,0,1,0
259145,0,0,23
259148,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259125,0,0,10
1,259138,1,12,2
2,259141,0,1,0
3,259145,0,0,23
4,259148,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259125,,,10
1,259138,1,12,2
2,259141,,1,
3,259145,,,23
4,259148,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC"
2,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC"
3,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC"
27,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC"
36,Mar 13 2023 11:32AM,259173,10,Emerginnova
37,Mar 13 2023 11:31AM,259172,10,"Virtue Rx, LLC"
38,Mar 13 2023 11:24AM,259171,10,"Methapharm, Inc."
45,Mar 13 2023 11:15AM,259169,10,Methapharm-G
46,Mar 13 2023 11:14AM,259168,18,Capsugel
47,Mar 13 2023 11:10AM,259166,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC",,,2
1,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC",,,1
2,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",,,24
3,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",,,9
4,Mar 13 2023 11:32AM,259173,10,Emerginnova,,,1
5,Mar 13 2023 11:31AM,259172,10,"Virtue Rx, LLC",,,1
6,Mar 13 2023 11:24AM,259171,10,"Methapharm, Inc.",,,7
7,Mar 13 2023 11:15AM,259169,10,Methapharm-G,,,1
8,Mar 13 2023 11:14AM,259168,18,Capsugel,,1,
9,Mar 13 2023 11:10AM,259166,10,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC",2,,
1,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC",1,,
2,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",24,,
3,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",9,,
4,Mar 13 2023 11:32AM,259173,10,Emerginnova,1,,
5,Mar 13 2023 11:31AM,259172,10,"Virtue Rx, LLC",1,,
6,Mar 13 2023 11:24AM,259171,10,"Methapharm, Inc.",7,,
7,Mar 13 2023 11:15AM,259169,10,Methapharm-G,1,,
8,Mar 13 2023 11:14AM,259168,18,Capsugel,,1,
9,Mar 13 2023 11:10AM,259166,10,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC",2,,
1,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC",1,,
2,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",24,,
3,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",9,,
4,Mar 13 2023 11:32AM,259173,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC",2.0,NaN,NaN
1,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",24.0,NaN,NaN
3,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",9.0,NaN,NaN
4,Mar 13 2023 11:32AM,259173,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 12:07PM,259179,19,"Snap Medical Industries, LLC",2.0,0.0,0.0
1,Mar 13 2023 12:07PM,259178,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",24.0,0.0,0.0
3,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",9.0,0.0,0.0
4,Mar 13 2023 11:32AM,259173,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4664886,67.0,28.0,0.0
15,518300,34.0,0.0,0.0
18,518346,1.0,1.0,0.0
19,1295773,24.0,18.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4664886,67.0,28.0,0.0
1,15,518300,34.0,0.0,0.0
2,18,518346,1.0,1.0,0.0
3,19,1295773,24.0,18.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,67.0,28.0,0.0
1,15,34.0,0.0,0.0
2,18,1.0,1.0,0.0
3,19,24.0,18.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,15,Released,34.0
2,18,Released,1.0
3,19,Released,24.0
4,10,Executing,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19
Status,,,,
Completed,0.0,0.0,0.0,1.0
Executing,28.0,0.0,1.0,18.0
Released,67.0,34.0,1.0,24.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19
0,Completed,0.0,0.0,0.0,1.0
1,Executing,28.0,0.0,1.0,18.0
2,Released,67.0,34.0,1.0,24.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19
0,Completed,0.0,0.0,0.0,1.0
1,Executing,28.0,0.0,1.0,18.0
2,Released,67.0,34.0,1.0,24.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()